# Siddhant Thakur - 19A1HP054

## Question 4: Mareket Basket Analysis (Apriori Algorithm)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
%matplotlib inline
from sklearn import metrics

In [2]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

df = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')
df = df[~df['InvoiceNo'].str.contains('C')]

In [4]:
basket = (df[df['Country'] =="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [5]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1)

In [6]:
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)

In [7]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(10 COLOUR SPACEBOY PEN),(CARD DOLLY GIRL),0.030612,0.030612,0.010204,0.333333,10.888889,0.009267,1.454082
1,(CARD DOLLY GIRL),(10 COLOUR SPACEBOY PEN),0.030612,0.030612,0.010204,0.333333,10.888889,0.009267,1.454082
2,(CHARLOTTE BAG SUKI DESIGN),(10 COLOUR SPACEBOY PEN),0.043367,0.030612,0.010204,0.235294,7.686275,0.008877,1.267661
3,(10 COLOUR SPACEBOY PEN),(CHARLOTTE BAG SUKI DESIGN),0.030612,0.043367,0.010204,0.333333,7.686275,0.008877,1.434949
4,(10 COLOUR SPACEBOY PEN),(ICE CREAM SUNDAE LIP GLOSS),0.030612,0.045918,0.010204,0.333333,7.259259,0.008798,1.431122


In [8]:
rules[ (rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
28,(12 COLOURED PARTY BALLOONS),(SET OF 9 BLACK SKULL BALLOONS),0.015306,0.066327,0.012755,0.833333,12.564103,0.011740,5.602041
49,(12 PENCILS TALL TUBE WOODLAND),(12 PENCILS TALL TUBE RED RETROSPOT),0.015306,0.022959,0.012755,0.833333,36.296296,0.012404,5.862245
68,(36 PENCILS TUBE WOODLAND),(36 PENCILS TUBE RED RETROSPOT),0.022959,0.045918,0.020408,0.888889,19.358025,0.019354,8.586735
142,(BINGO SET),(4 TRADITIONAL SPINNING TOPS),0.010204,0.071429,0.010204,1.000000,14.000000,0.009475,inf
263,(ALARM CLOCK BAKELIKE CHOCOLATE),(ALARM CLOCK BAKELIKE PINK),0.028061,0.102041,0.022959,0.818182,8.018182,0.020096,4.938776
...,...,...,...,...,...,...,...,...,...
1139910,"(DOLLY GIRL CHILDRENS CUP, JUMBO BAG APPLES)","(LUNCH BOX WITH CUTLERY RETROSPOT, LUNCH BAG A...",0.010204,0.010204,0.010204,1.000000,98.000000,0.010100,inf
1139939,"(LUNCH BOX I LOVE LONDON, SKULL LUNCH BOX WITH...","(LUNCH BOX WITH CUTLERY RETROSPOT, JUMBO BAG A...",0.010204,0.010204,0.010204,1.000000,98.000000,0.010100,inf
1139940,"(SKULL LUNCH BOX WITH CUTLERY, DOLLY GIRL CHIL...","(LUNCH BOX WITH CUTLERY RETROSPOT, JUMBO BAG A...",0.010204,0.010204,0.010204,1.000000,98.000000,0.010100,inf
1139952,"(ALARM CLOCK BAKELIKE PINK, DOLLY GIRL CHILDRE...","(LUNCH BOX WITH CUTLERY RETROSPOT, JUMBO BAG A...",0.010204,0.010204,0.010204,1.000000,98.000000,0.010100,inf


In [9]:
print(basket['ALARM CLOCK BAKELIKE GREEN'].sum())
print(basket['ALARM CLOCK BAKELIKE RED'].sum())

340.0
316.0


## how the combinations vary by country of purchase

In [10]:
basket2 = (df[df['Country'] =="Germany"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket_sets2 = basket2.applymap(encode_units)
basket_sets2.drop('POSTAGE', inplace=True, axis=1)
frequent_itemsets2 = apriori(basket_sets2, min_support=0.05, use_colnames=True)
rules2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=1)

rules2[ (rules2['lift'] >= 4) &
        (rules2['confidence'] >= 0.5)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN WOODLAND ANIMALS),0.115974,0.137856,0.067834,0.584906,4.242887,0.051846,2.076984
6,(PLASTERS IN TIN SPACEBOY),(PLASTERS IN TIN WOODLAND ANIMALS),0.107221,0.137856,0.061269,0.571429,4.145125,0.046488,2.011670
10,(RED RETROSPOT CHARLOTTE BAG),(WOODLAND CHARLOTTE BAG),0.070022,0.126915,0.059081,0.843750,6.648168,0.050194,5.587746
